In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

In [ ]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from IPython.display import Markdown
from plotly.offline import init_notebook_mode, iplot
from scipy import stats

init_notebook_mode()

from src.report import (
    conf_int,
    t_test,
    f_test,
    general,
    general_assessment,
    other,
    dw_test,
    bg_test
)
from src.stats import LinearRegression, add_constant

In [ ]:
from data import DATA_FRAME

# Исходные данные

In [ ]:
DATA_FRAME

# Матрица парной корреляции

In [ ]:
DATA_FRAME.corr()

# Диаграммы рассеивания

In [ ]:
iplot(px.scatter(DATA_FRAME, y='y', x='x_1', trendline='ols', trendline_color_override='orange'))

In [ ]:
iplot(px.scatter(DATA_FRAME, y='y', x='x_3', trendline='ols', trendline_color_override='orange'))

# Парная линейная регрессия

## Python

In [ ]:
Y = DATA_FRAME['y']
X = add_constant(DATA_FRAME['x_1'])

In [ ]:
paired_model = LinearRegression(Y, X)

In [ ]:
print(paired_model.summary())

### Модель

In [ ]:
general(paired_model)

### Общая оценка

In [ ]:
general_assessment(paired_model)

### Оценка модели на значимость

In [ ]:
f_test(paired_model)

### Оценка значимости параметров модели

In [ ]:
t_test(paired_model)

### Доверительные интервалы

In [ ]:
conf_int(paired_model)

### Дополнительные оценки

In [ ]:
other(paired_model)

### Автокорреляция

In [ ]:
resid = paired_model.results.resid

In [ ]:
iplot(px.scatter(
    x=resid.shift(-1),
    y=resid,
    labels={
        'x': 'e_{i-1}',
        'y': 'e'
    }
))

In [ ]:
dw_test(paired_model)

In [ ]:
bg_test(paired_model)

In [ ]:
bg_critical, bg_pvalue = sm.stats.acorr_breusch_godfrey(paired_model.results, nlags=1)[:2]

In [ ]:
Markdown(fr"""
$ BG = {bg_critical} $<br>
$ p\text{{-}}value = {bg_pvalue} $<br>
<br>
$ {bg_pvalue:.3f} > {0.05} $<br>
$ p\text{{-}}value > \alpha \rightarrow $ гипотеза $ H_0 $ <b>принимается</b> - автокорреляция <b>отсутствует</b>.
""")

### Гетероскедастичность

In [ ]:
xs_resid_df = pd.concat([paired_model.x, resid.rename('e')], axis=1)
x_sorted = xs_resid_df.sort_values(['x_1'])
iplot(px.scatter(
    x=x_sorted['x_1'],
    y=(x_sorted['e'] ** 2),
    labels={'x': 'x', 'y': 'e^2'}
))

In [ ]:
exog = paired_model.results.model.exog
endog = paired_model.results.model.endog

In [ ]:
Markdown(fr"""
### Тест Голдфельда-Квандта
<br><br>
$
H_0: S^2 = S_1^2 = ... = S_n^2 , \\
H_1: S_i^2 = ... = S^2 x_{{ij}}^2 .
$
<br><br>
$ GQ = \frac{{\sum{{e_{{i пред. большая}}^2}}}}{{\sum{{e_{{i пред. меньшая}}^2}}}} $
""")

In [ ]:
gq_critical, gq_pvalue = sm.stats.het_goldfeldquandt(endog, exog, idx=1)[:2]

In [ ]:
Markdown(fr"""
$ GQ = {gq_critical} $<br>
$ p\text{{-}}value = {gq_pvalue} $<br>
<br>
$ {gq_pvalue:.3f} < {0.05} $<br>
$ p\text{{-}}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - гетероскедастичность <b>значима</b>.
""")

In [ ]:
Markdown(fr"""
### Тест Бреуша-Пагана
<br><br>
$
H_0: S^2 = S_1^2 = ... = S_n^2 , \\
H_1: S_i^2 = ... = S^2 (x_1, ..., x_k).
$
<br><br>
$ BP = ? $
""")

In [ ]:
bp_critical, bp_pvalue = sm.stats.het_breuschpagan(resid, exog)[2:4]

In [ ]:
Markdown(fr"""
$ BP = {bp_critical} $<br>
$ p\text{{-}}value = {bp_pvalue} $<br>
<br>
$ {bp_pvalue:.3f} > {0.05} $<br>
$ p\text{{-}}value > \alpha \rightarrow $ гипотеза $ H_0 $ <b>принимается</b> - гетероскедастичность <b>незначима</b>.
""")
